In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/perrin-freres-monthly-champagne-sales/Perrin Freres monthly champagne sales millions.csv')
df

In [ ]:
df.columns = ["Month","Sales"]
df

In [ ]:
df.drop(106,axis=0,inplace=True)#Dropping 106th row since it has Nan value
df.drop(105,axis=0,inplace=True)

In [ ]:
df.dtypes

In [ ]:
df['Month']=pd.to_datetime(df['Month'])
df.head()

In [ ]:
df.set_index('Month',inplace=True)#???
df.head()

In [ ]:
df.describe()

**##2. Visualize the Data**

In [ ]:
df.plot()# It is showing seasonal format since some particular time it is high and other time it is low.

In [ ]:
# Testing for Stationarity
from statsmodels.tsa.stattools import adfuller
test_result = adfuller(df['Sales'])
#H0 : It is non stationary
#H1: It is Stationary
def adfuller_test(sales):
    result=adfuller(sales)
    labels=['ADF Test Statistic','p-value','Lags used','Number of Observation used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value))
    if result[1] <= 0.05:
        print('Strong evidence against Null hypothesis(H0), reject null hypothesis, Data is stationary')
    else:
        print('Weak evidence against Null hypothesis(H1), timeseries has unit root indicating non stationarity')

In [ ]:
adfuller_test(df['Sales'])

Differencing to remove nonstationarity

In [ ]:
df['Sales First difference']= df['Sales']-df['Sales'].shift(1)
df['Sales'].shift(1)

In [ ]:
# But one problem here arises is the data comtains seasonality which repeats every 12 months. So to remove the seasonality we shift by 12 months.
df['Seasonal First difference']= df['Sales']-df['Sales'].shift(12)
df['Seasonal First difference']

In [ ]:
adfuller_test(df['Sales First difference'].dropna())

In [ ]:
adfuller_test(df['Seasonal First difference'].dropna())

In [ ]:
df['Seasonal First difference'].plot()

So THe data becomes Stationary

In [ ]:
### Entering into Auto Regressive Modelling

In [ ]:

from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df['Sales'])
plt.show()

In [ ]:
import statsmodels as sm

from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig= plt.figure(figsize=(12,8))
ax1=fig.add_subplot(211)
fig = sm.graphics.tsaplots.plot_acf(df['Seasonal First difference'].iloc[13:],lags=40,ax=ax1)
ax2=fig.add_subplot(212)
fig = sm.graphics.tsaplots.plot_pacf(df['Seasonal First difference'].iloc[13:],lags=40,ax=ax2)


In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(df['Sales'],order=(1,1,1))
model_fit=model.fit()
model_fit.summary()
#But since data is seasonal it will not work well

In [ ]:
import statsmodels.api as sm
modelsnsnl = sm.tsa.statespace.SARIMAX(df['Sales'],order=(1,1,1),seasonal_order=(1,1,1,12))
result=modelsnsnl.fit()

In [ ]:
df['forecast']=result.predict(start=90,end=103,dynamic=True)
df[['Sales','forecast']].plot(figsize=(12,8))

In [ ]:

from pandas.tseries.offsets import DateOffset
future_dates=[df.index[-1]+ DateOffset(months=x)for x in range(0,24)]
future_datest_df=pd.DataFrame(index=future_dates[1:],columns=df.columns)

future_datest_df

In [ ]:
future_df=pd.concat([df,future_datest_df])

future_df['forecast'] = result.predict(start = 104, end = 120, dynamic= True)  
future_df[['Sales', 'forecast']].plot(figsize=(12, 8))

In [ ]:
# So this is the Sales Forecast